In [ ]:
# Use the mlflow tracking server where you trained your models in previous steps
# %load_ext dotenv
# %dotenv

## Check experiment runs and get the best run

In [ ]:
import os
import mlflow
from mlflow.tracking import MlflowClient
from sklearn import datasets
import numpy as np

In [ ]:
# Create an instance of the MflowClient, we will use it to query our tracking server
client = MlflowClient()

In [ ]:
#Get the experiment
experiment = mlflow.get_experiment(experiment_id="<experiment-id>")

In [ ]:
# Get experiment runs
runs = client.search_runs(experiment_ids = [experiment.experiment_id])

In [ ]:
#Choose a metric to get the best run
metrics = [run.data.metrics["mse"] for run in runs]
best_rmse = min(metrics)

best_run = [run for run in runs if run.data.metrics["mse"] == best_rmse][0]
best_run.info

In [ ]:
# Register the model from the best run
model_name = "my best model"

model_uri = f"runs:/{best_run.info.run_id}/model"
model_version = mlflow.register_model(model_uri = model_uri, name = model_name)

## Load the model

In [ ]:
model_info = client.get_registered_model(name = model_name)

In [ ]:
model_info

In [ ]:
model = mlflow.sklearn.load_model(model_uri)

## Running inference

In [ ]:
diabetes = datasets.load_diabetes()
data = diabetes.data
preds = model.predict(data)

## Adding metadata to our model

In [ ]:
client.set_model_version_tag(model_name, model_version.version, key = "prediction_works", value = "true")

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)

In [ ]:
model_version_info.tags

## Promoting our model to Staging

We are happy with our model (for now), so let's promote it to Staging

In [ ]:
client.transition_model_version_stage(model_name, model_version.version, stage = "Staging")

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)

In [ ]:
# check if it worked
model_version_info.current_stage